In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl

import math 

from bin import *

from IPython.display import clear_output
%load_ext autoreload
%autoreload 2

In [2]:
mpl.use("pgf")

mpl.rcParams.update({
    "pgf.texsystem": "lualatex",
    'font.family': 'serif',
    'font.size' : 8,
    'text.usetex': True,
    'pgf.rcfonts': False,
    'pgf.preamble': tex_preamble,
})

In [3]:
n = 300
Λ = model_problem_spectrum(n,.8,1e3)

In [4]:
M = (0,0,1)
N = (1,0,1)

In [5]:
A = np.diag(Λ)

b_vec = np.ones(n,dtype=np.double)
b_vec /= np.linalg.norm(b_vec)

In [6]:
K = 90
K_ro = 45
d = 4
Q,(a_,b_) = lanczos_reorth(A,b_vec,K+d,reorth=0)
Q_ro,(a_ro_,b_ro_) = lanczos_reorth(A,b_vec,K_ro+d,reorth=K_ro+d)

T = np.diag(a_) + np.diag(b_[:-1],-1) + np.diag(b_[:-1],1)
T_ro = np.diag(a_ro_) + np.diag(b_ro_[:-1],-1) + np.diag(b_ro_[:-1],1)

e0 = np.zeros(K+d)
e0[0] = 1

In [7]:
M_func = lambda x: (M[0]*x**2+M[1]*x+M[2])
N_func = lambda x: (N[0]*x**2+N[1]*x+N[2])
r = lambda x: M_func(x)/N_func(x)

rAb = r(Λ)*b_vec
Hsqrt = np.diag(N_func(Λ)**.5)

In [8]:
error_OR = np.full(K+d,np.nan)
error_OR_ro = np.full(K_ro+d,np.nan)

x_OR = np.full((n,K+d),np.nan)
x_OR_ro = np.full((n,K_ro+d),np.nan)

for i in range(1,K+d):

    lan_lm = streaming_banded_rational(n,i,M,N)
    for j in range(i):
        lan_lm.read_stream(Q[:,j],a_[j],b_[j])

    lan_lm.finish_up()      

    x_OR[:,i] = lan_lm()    
    error_OR[i] = np.linalg.norm(Hsqrt*(rAb - x_OR[:,i]))


for i in range(1,K_ro+d):

    lan_lm = streaming_banded_rational(n,i,M,N)
    for j in range(i):
        lan_lm.read_stream(Q_ro[:,j],a_ro_[j],b_ro_[j])

    lan_lm.finish_up()      

    x_OR_ro[:,i] = lan_lm()
    error_OR_ro[i] = np.linalg.norm(Hsqrt*(rAb - x_OR_ro[:,i]))

In [9]:
bound_OR = np.full(K,np.nan)
bound_OR_ro = np.full(K_ro,np.nan)

for k in range(1,K):
    bound_OR[k] = 0
    for i in range(k,k+d):
        bound_OR[k] += np.linalg.norm(Hsqrt*(x_OR[:,i+1] - x_OR[:,i]))**2
        
for k in range(1,K_ro):
    bound_OR_ro[k] = 0
    for i in range(k,k+d):
        bound_OR_ro[k] += np.linalg.norm(Hsqrt*(x_OR_ro[:,i+1] - x_OR_ro[:,i]))**2

In [10]:
fig = plt.figure(figsize=(figure_width*cm,figure_height*cm))

ax = fig.add_axes([left, bottom, width, height])

ax.plot(np.sqrt(bound_OR),**line_styles['l2'])
ax.plot(error_OR,**line_styles['l1'])

ax.plot(np.sqrt(np.abs(bound_OR_ro)),**line_styles['l4'])
ax.plot(error_OR_ro,**line_styles['l3'])

ax.grid(True,linestyle=':',linewidth=.5)

ax.set_yscale('log')
ax.set_ylim(1e-14,1e1)

ax.set_xlabel('number of matvecs: $k$')

plt.savefig('imgs/ch5_OR_error_est.pdf')
plt.close()